In [2]:
import hoomd
from hoomd import md

# Advancing simulations

Use `run` to advance a simulation forward a given number of time steps. For this tutorial, we use the same setup from the Quick Start.

In [2]:
hoomd.context.initialize('--mode=cpu')
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5)
nl = md.nlist.cell()
lj = md.pair.lj(r_cut=3.0, nlist=nl)
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)
all = hoomd.group.all();
md.integrate.mode_standard(dt=0.001)
md.integrate.brownian(group=all, kT=0.1, seed=987)

HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 125 particles
notice(2): integrate.langevin/bd is using specified gamma values


After initialization with `create_lattice`, the simulation starts at time step 0. Query the current step counter with [get_step](http://hoomd-blue.readthedocs.io/en/stable/package-hoomd.html#hoomd.get_step).

In [3]:
hoomd.get_step()

0

Each HOOMD script can have one **integrator** active at a time. This example uses the `mode_standard` integrator from the `md` package. On each time step, `mode_standard` advances the simulation forward `dt` time units, integrating the equations of motion specified in the provided integration methods (brownian dynamics in this example). Use `run` to advance the simulation a given number of steps.

In [4]:
hoomd.run(1000)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 125
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:00 | Step 1000 / 1000 | TPS 1592.64 | ETA 00:00:00
Average TPS: 1591.18
---------
-- Neighborlist stats:
54 normal updates / 10 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 22 / n_neigh_avg: 9.392
shortest rebuild period: 12
-- Cell list stats:
Dimension: 2, 2, 2
n_min    : 11 / n_max: 19 / n_avg: 15.625
** run complete **


Now we are at step 1000.

In [5]:
hoomd.get_step()

1000

## Run options

The output from run is useful in batch jobs. However, sometimes you may want to supress it with the `quiet` option.

In [6]:
hoomd.run(1000, quiet=True)
hoomd.get_step()

2000

You may want to limit a run to a given length of time (e.g. for short benchmark runs). Use `limit_hours` to stop a run after so many hours, even if it has not reached the requested number of time steps.

In [7]:
hoomd.run(1e6, limit_hours=5/3600, quiet=True)

notice(2): Ending run at time step 49159 as 0.00138889 hours have passed


The profile option prints information on how much time was spent in each portion of the calculation. This is helpful when trying to identify why a simulation is running slowly.

In [8]:
hoomd.run(10000, profile=True)

** starting run **
Time 00:00:06 | Step 59159 / 59159 | TPS 8646.26 | ETA 00:00:00
Average TPS: 8634.3
Simulation:  1.1583s | 100.000% 
        Integrate:  0.3397s | 29.332% 
                BD step 1:  0.3143s | 27.138% 
                Net force:  0.0191s |  1.648% 
                Self:       0.0063s |  0.546% 
        Neighbor:   0.1010s |  8.721% 
                Cell:        0.0029s |  0.247% 
                        compute:  0.0026s |  0.226% 
                        init:     0.0000s |  0.002% 
                Dist check:  0.0114s |  0.984% 
                compute:     0.0838s |  7.234% 
                head-list:   0.0001s |  0.004% 
                Self:        0.0029s |  0.251% 
        Pair lj:    0.7056s | 60.921% 
        SFCPack:    0.0024s |  0.203% 
        Self:       0.0095s |  0.822% 
---------
-- Neighborlist stats:
262 normal updates / 101 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 55 / n_neigh_avg: 24.20800
shortest rebuild period: 12
--

## Callbacks

You can specify an arbitrary python callback to run. This function will be called at the given period. If the function returns a negative value, the run is aborted.

In [9]:
def my_callback(step):
    print("This is step", step)
    if step >= 70000:
        return -1
    else:
        return 0

hoomd.run(1e6, callback=my_callback, callback_period=1000, quiet=True)

This is step 60000
This is step 61000
This is step 62000
This is step 63000
This is step 64000
This is step 65000
This is step 66000
This is step 67000
This is step 68000
This is step 69000
This is step 70000
notice(2): End of run requested by python callback at step 70000 / 1059159
